In [1]:
import requests
from bs4 import BeautifulSoup


headers = {
    'User-Agent': (
        'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) '
        'AppleWebKit/537.36 (KHTML, like Gecko) '
        'Chrome/72.0.3626.121 Safari/537.36'
    ),
}


def get_like_count(song_no_list):
    api_url = "https://www.melon.com/commonlike/getSongLike.json"
    params = {"contsIds": song_no_list}
    res = requests.get(api_url, params=params, headers=headers)
    res.raise_for_status()
    response = res.json()
    like_list = response["contsLike"]
    like_dict = {
        str(song["CONTSID"]): song["SUMMCNT"]
        for song in like_list}
    return like_dict


def get_song_list():
    res = requests.get("http://www.melon.com/chart/index.htm", headers=headers)
    res.raise_for_status()
    html = res.text
    soup = BeautifulSoup(html, 'html.parser')
    tr_tag_list = soup.select('.d_song_list tbody tr')

    song_list = []

    for rank, tr_tag in enumerate(tr_tag_list, 1):
        song_no = tr_tag["data-song-no"]
        song_tag = tr_tag.select_one('a[href*=playSong]')
        album_tag = tr_tag.select_one('.wrap_song_info a[href*=goAlbumDetail]')
        artist_tag = tr_tag.select_one('a[href*=goArtistDetail]')
        
        song = {
            'song_no': song_no,
            'title': song_tag.text,
            'album': album_tag.text,
            'artist': artist_tag.text,
            'rank': rank,
        }
        song_list.append(song)

    song_no_list = [song["song_no"] for song in song_list]
    like_dict = get_like_count(song_no_list)

    for song in song_list:
        like_count = like_dict[song["song_no"]]
        song["like"] = like_count

    return song_list

In [25]:
melon_top100_list = get_song_list()
melon_top100 = get_song_list()

# `"방탄소년단"` 의 곡명만 출력하는 코드

In [151]:
song_filter = filter(
    lambda song:song["artist"] == "방탄소년단",
    melon_top100
)

song_map = map(
    lambda song:song["title"],
    song_filter
)

list(song_map)

['Dynamite',
 'Life Goes On',
 '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)',
 '내 방을 여행하는 법',
 '봄날',
 'Blue & Grey',
 '잠시',
 'ON',
 'Stay',
 '병']

# 2. 곡명에 "가을"이 들어가는 곡명만 출력하는 코드
    - Hint: 포함여부 = "가을" in 곡명

In [153]:
song_filter = filter(
    lambda song:"가을" in song["title"],
    melon_top100
)

song_map = map(
    lambda song:song["title"],
    song_filter
)

list(song_map)

['가을밤에 든 생각']

# 3. 좋아요 수가 200,000이 넘는 곡수를 구하는 코드
    - Hint: int(좋아요) > 200000

In [154]:
song_filter = filter(
    lambda song:song["like"] > 200_000,
    melon_top100
)

song_map = map(
    lambda song:song["title"],
    song_filter
)

list(song_map)

['Dynamite',
 '오래된 노래',
 '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지',
 '에잇(Prod.&Feat. SUGA of BTS)',
 '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야',
 '아로하',
 'METEOR',
 'Blueming',
 '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)',
 '모든 날, 모든 순간 (Every day, Every Moment)',
 '오늘도 빛나는 너에게 (To You My Light) (Feat.이라온)',
 '봄날',
 '2002',
 '다시 여기 바닷가',
 '너를 만나',
 '아무노래',
 'ON',
 'Love poem',
 'Paris In The Rain']

# 4. 가수 별 곡수를 출력하는 코드
    - Hint: `dict` 자료구조 활용

In [156]:
from collections import Counter

Counter(map(
    lambda song:song["artist"],
    melon_top100
))

Counter({'미란이': 2,
         '방탄소년단': 10,
         '장범준': 2,
         '임창정': 2,
         'BLACKPINK': 4,
         '산들': 1,
         '경서': 1,
         '릴보이 (lIlBOI)': 2,
         '스탠딩 에그': 1,
         'Jawsh 685': 1,
         '환불원정대': 1,
         '규현 (KYUHYUN)': 1,
         '마마무 (Mamamoo)': 2,
         '박진영': 1,
         'AKMU (악동뮤지션)': 2,
         'Mariah Carey': 2,
         '아이유': 4,
         '이승기': 1,
         '스윙스': 1,
         '적재': 1,
         'TWICE (트와이스)': 1,
         '오반': 2,
         '노을': 2,
         '잔나비': 1,
         '오마이걸 (OH MY GIRL)': 2,
         '조정석': 1,
         '제시 (Jessi)': 1,
         '화사 (Hwa Sa)': 1,
         '창모 (CHANGMO)': 1,
         '폴킴': 4,
         '백지영': 1,
         '순순희': 1,
         'Ariana Grande': 1,
         '벤': 1,
         '이하이': 1,
         'Maroon 5': 1,
         'Tones And I': 1,
         '경서예지': 1,
         '전상근': 1,
         '블루 (BLOO)': 1,
         '마크툽 (MAKTUB)': 2,
         'Crush': 1,
         '신예영': 1,
         '허성현 (Rose de Penny)': 1,
  

# 5. 좋아요 수로 TOP10의 가수와 곡명, 좋아요수 출력하는 코드
    - Hint : 리스트에는 sort 함수가 제공되고 있으며, 빌트인 함수에는 `sorted`가 제공됩니다.

In [165]:
# 1. 좋아요 순서 sort
# 2. Top10 - [artist,title,like]

song_sorted = sorted(melon_top100, key=lambda song:song["like"], reverse=True)[:10]
song_map = map(
    lambda song:(song["artist"],song["title"],song["like"]),
    song_sorted
)
list(song_map)

[('방탄소년단', '봄날', 516607),
 ('방탄소년단', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)', 406735),
 ('방탄소년단', 'Dynamite', 362539),
 ('폴킴', '모든 날, 모든 순간 (Every day, Every Moment)', 352595),
 ('Anne-Marie', '2002', 344471),
 ('아이유', '에잇(Prod.&Feat. SUGA of BTS)', 331240),
 ('AKMU (악동뮤지션)', '어떻게 이별까지 사랑하겠어, 널 사랑하는 거지', 325007),
 ('아이유', 'Love poem', 277585),
 ('아이유', 'Blueming', 276945),
 ('폴킴', '너를 만나', 275108)]

In [4]:
#------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------#
#------------------------------------------------------------------------------------------------#

# f 객체에 대해서 슬라이싱 적용하기

In [168]:
from itertools import islice 

with open ("20201202/melon_top100.csv", "rt") as f:
    for index, value in islice(enumerate(f),5):
        print(f"[{index}] {value}")

[0] ,song_no,title,album,artist,rank,like

[1] 0,33077590,VVS (Feat. JUSTHIS) (Prod. GroovyRoom),쇼미더머니 9 Episode 1,미란이,1,96325

[2] 1,32872978,Dynamite,Dynamite (DayTime Version),방탄소년단,2,360973

[3] 2,33013877,잠이 오질 않네요,잠이 오질 않네요,장범준,3,82843

[4] 3,33077234,Life Goes On,BE,방탄소년단,4,151888



# 1. 유효한 팰린드롬

주어진 문자열이 팰린드롬인지 확인하세요. 대소문자를 구별하지 않으며, `영문자와 숫자` 만으로 대상으로 합니다.

## 팰린드롬이란?

앞뒤가 똑같은 단어가 문장으로, 뒤집어도 같은 말이 되는 단어 또는 문장을 팰린드롬(Palindrome)이라고 합니다. 우리말로는 "회문"이라 부르며, 문장 중에서는 대표적으로 "소주 만 병만 주소" 같은 문장이 이에 해당합니다. 이 문장은 뒤집어도 "소주 만 병만 주소"가 됩니다. 이러한 팰린드롬의 특징을 응용하면 여러 가지 재미있는 문제들을 많이 만들어낼 수 있기 때문에 코딩 테스트에서 매우 자주 출제되는 주제입니다.

In [173]:
import re

def is_palindrome(s):
    # TODO: 구현해주세요.
    value = re.sub(r"[^a-zA-Z\d+]","",s).lower()
    re_value = value[::-1]
    
    if value != re_value:
        return False
    return True

print(is_palindrome("A man, a plan, a canal: Panama"))  # 예상결과: True
print(is_palindrome("race a car"))  # 예상결과: False?

True
False


# 2. 가장 흔한 단어

금지된 단어를 제외한 가장 흔하게 등장하는 단어를 출력하세요. 대소문자 구분을 하지 않으며, 구두점(마침표, 쉼표 등) 또한 무시합니다.

In [188]:
import re
from collections import Counter

문장 = "Bob hit a ball, the hit BALL flew far after it was hit."
금지어_리스트 = ["hit"]

value = re.sub(r"[,\.]","",문장).lower().split()
filter_vlaue = filter(
    lambda x:x not in 금지어_리스트,
    value
)


return_value,_ = Counter(filter_vlaue).most_common()[0]
return_value

# 예상결과: 'ball'

'ball'

# 3. 그룹 애너그램

문자열 배열을 받아서, 애너그램 단위로 그룹핑하세요. 출력되는 순서는 상관없습니다.

입력

```python
["eat", "tea", "tan", "ate", "nat", "bat"]
```

출력

```python
[
    ["ate", "eat", "tea"],
    ["nat", "tan"],
    ["bat"],
]
```

In [96]:
word_list = ["eat", "tea", "tan", "ate", "nat", "bat"]

In [198]:
# list를 기본으로 가지는 dict
from collections import defaultdict

return_list = defaultdict(list)
for word in word_list:
    key = "".join(sorted(word))
    return_list[key].append(word)

list(return_list.values())

[['eat', 'tea', 'ate'], ['tan', 'nat'], ['bat']]

# 4. 인자 갯수에 상관없이, 지정 함수의 실행시간을 출력하는 장식자


```python
import time

def check_runtime(fn):
    # TODO: ...

@check_runtime
def mysum(x, y):
    time.sleep(2)
    return x + y

mysum(1, 2)
```

출력 : `약 2초가 소요`

time.time()

In [201]:
import time

def check_runtime(fn):
    # TODO: ...
    def wrap(*args):
        start_time = time.time()
        try:
            return fn(*args)
        finally:
            end_time = time.time()
            print(f"약 {end_time - start_time}초가 소요")
    return wrap

@check_runtime
def mysum(x, y):
    time.sleep(2)
    return x + y

mysum(1, 2)

약 2.0042552947998047초가 소요


3

# 5. Memoization

다음의 함수들이 빠르게 동작할 수 있도록 `@memoize` 장식자를 함수버전과 클래스버전으로 만들어주세요. `RuntimeError: maximum recursion depth exceeded` 예외가 발생한다면 이 예외가 발생하지 않도록 설정해주세요.

In [208]:
import sys
# RuntimeError: maximum recursion depth exceeded 예외가 발생하지 않도록 limit을 늘립니다.
sys.setrecursionlimit(10000)

# 함수버전, 클래스버전
def memoize(fn):
    cached = {}
    def wrap(n):
        key = n
        if key not in cached:
            cached[key] = fn(n)
        return cached[key]
    return wrap

class Memoize:
    def __init__(self, fn):
        self.fn = fn
        self.cached = {}
        
    def __call__(self, n):
        key = n
        if key not in self.cached:
            self.cached[key] = self.fn(n)
        return self.cached[key]

@memoize
def fib1(n):
    if n in (1, 2):
        return 1
    return fib1(n-1) + fib1(n-2)

@Memoize
def fib2(n):
    if n in (1, 2):
        return 1
    return fib2(n-1) + fib2(n-2)

print(fib1(1000))    # 1초 내외
print(fib1(1000))    # 1초 내외
print(fib1(1000))    # 1초 내외
print(fib2(1000))    # 1초 내외
print(fib2(1000))    # 1초 내외
print(fib2(1000))    # 1초 내외

43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875
43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875
43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875
43466557686937456435688527675040625802564660517371780402481729089536555417949051890403879840079255169295922593080322634775209689623239873322471161642996440906533187938298969649928516003704476137795166849228875
4346655768693745643568852767504062580256466051737178040248172908953655541794905189040387984007925516929592259308032263477520968962323987332247116164299644090653